In [1]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import os

In [2]:
def check_duplicate_columns(df):
    duplicate_columns = df.columns[df.columns.duplicated()]
    return duplicate_columns

In [3]:
df_ov_clinical = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_Zhang_TCGA_CPTAC_OV/mmc2.xlsx")
df_ov_clinical.head(10)
len(df_ov_clinical)

/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


174

In [4]:
selected_stages= ["IIIA","IIIB","IIIC","IV"]
df_ov_clinical = df_ov_clinical[df_ov_clinical["tumor_stage"].isin(selected_stages)]

In [5]:
df_ov_clinical["PlatinumStatus"].value_counts()
df_ov_clinical = df_ov_clinical[df_ov_clinical['PlatinumStatus'] != "Not available"]

# remove not avalable... 
df_ov_clinical['label'] = df_ov_clinical['PlatinumStatus'].map({'Sensitive': 1, 'Resistant': 0})
df_ov_clinical['label'].value_counts()

1    85
0    35
Name: label, dtype: int64

#### Load proteomic data

In [6]:
tcga_proteomic = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_Zhang_TCGA_CPTAC_OV/mmc3-2.xlsx")

In [7]:
tcga_prots = tcga_proteomic["hgnc_symbol"].to_list()

In [8]:
# comapre to training dataset here in analysis
# load other to find amount of overlapping prot and phospoprots.
gloabal_prot = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_processed_data/FFPE_discovery_globalprotein_imputed.tsv",delimiter='\t')
HGSOC_prots = gloabal_prot["HGNC symbol"].to_list()

In [9]:
HGSOC_prots_set = ['ABCB8', 'ACADSB', 'ACOT7', 'AIFM1', 'ARNT', 'ATM',
       'ATP6V1D', 'BCL2L1', 'BIRC2', 'CA2', 'CAMK2D', 'CARMIL1', 'CASP7',
       'CCDC167', 'CDKN1A', 'CDKN1B', 'CHUK', 'COX6A1', 'COX6C', 'COX7A2',
       'COX7C', 'CPOX', 'EGFR', 'FDX1', 'GLUD1', 'GOLPH3L', 'GPX1', 'GPX4',
       'HACL1', 'HADH', 'IDH1', 'IDI1', 'IL4I1', 'KEAP1', 'LDHA', 'LGALS1',
       'LTA4H', 'MTDH', 'MYD88', 'NBN', 'NCAPH2', 'NDUFB3', 'NFKB1', 'NFKB2',
       'PDK1', 'PECR', 'PLIN2', 'PTGES2', 'RAB25', 'RELA', 'RHOA', 'RIPK1',
       'RPS6KB2', 'S100A10', 'SENP1', 'STAT3', 'TALDO1', 'TFRC', 'TGM2',
       'TOP3B', 'TPMT', 'VCAM1', 'XIAP', 'YWHAG']


set1 = set(tcga_prots)
set2 = set(HGSOC_prots_set)

# Find intersection
intersection = set1 & set2

# Convert back to list
intersecting_list = list(intersection)

len(intersecting_list)
# 7621 intesecting.... 60/64 in dicsovery set... 

60

In [20]:
tcga_proteomic.head()


seperate both locations?.... or just take first client.... 

,refseq_peptide,hgnc_symbol,PNNL-TCGA-09-1664,PNNL-TCGA-13-1484,PNNL-TCGA-13-1488,PNNL-TCGA-13-1489,PNNL-TCGA-13-1494,PNNL-TCGA-13-1495,PNNL-TCGA-13-1499,PNNL-TCGA-13-2071,...,JHU-TCGA-61-1911,JHU-TCGA-61-1914,JHU-TCGA-61-1915,JHU-TCGA-61-1918,JHU-TCGA-61-1995,JHU-TCGA-61-2008,JHU-TCGA-61-2087,JHU-TCGA-61-2088,JHU-TCGA-61-2094,JHU-TCGA-61-2613
0,NP_000005,A2M,-0.0581,-0.688,0.5180,-0.432,0.333,0.109,-1.230,0.172,...,0.265750,-0.688250,-0.071250,-0.767250,-0.821250,-0.360550,-1.022250,-1.172250,-0.132250,1.165750
1,NP_000007,ACADM,-0.1040,0.404,0.0714,0.272,-0.369,0.175,-0.295,-0.425,...,0.073225,-0.326775,-1.069775,-0.383775,0.105225,0.365225,-0.277775,0.401225,0.056225,-0.372775
2,NP_000008,ACADS,0.1150,-0.045,0.5500,0.143,-0.209,-0.616,0.108,-0.346,...,0.096885,-0.131685,1.132315,0.850315,0.798315,-0.789685,-0.296685,0.529315,0.667315,0.876315
3,NP_000009,ACADVL,0.1850,-0.147,0.2540,1.090,-0.220,-0.395,-0.110,-0.221,...,1.001100,-0.027900,-0.017900,0.270800,-0.265900,-0.084900,0.443100,0.699100,0.328100,0.499100
4,NP_000010,ACAT1,0.0699,0.176,0.2770,0.968,-1.570,-0.506,-0.468,-0.284,...,-0.339050,0.221950,0.047950,0.847950,0.434950,0.219950,1.267950,0.735950,0.202950,-0.213050


# Norm , Impute and Merge proteomics onto clinical

In [21]:
from sklearn.impute import SimpleImputer, KNNImputer


# transopose and organise 
tcga_proteomic_t = tcga_proteomic.T
tcga_proteomic_t.columns = tcga_proteomic_t.iloc[1]
tcga_proteomic_t = tcga_proteomic_t.iloc[2:]




columns = pd.Series(tcga_proteomic_t.columns)
# Keep the first occurrence of the column name and discard the others
unique_columns = columns[~columns.duplicated(keep='first')].tolist()
print(len(unique_columns))

# Select only the unique columns
df_unique_columns = tcga_proteomic_t[unique_columns]


duplicate_columns = check_duplicate_columns(df_unique_columns)
if len(duplicate_columns) > 0:
    print("Duplicate columns found:", duplicate_columns.tolist())
else:
    print("No duplicate columns found.")


# impute 
# tcga_proteomic_t.index
knn_imputer = KNNImputer(n_neighbors=5)

scaler = StandardScaler()
# Fit and transform the DataFrame
tcga_proteomic_data = pd.DataFrame(scaler.fit_transform(tcga_proteomic_t), columns=tcga_proteomic_t.columns, index=tcga_proteomic_t.index)

tcga_proteomic_data = pd.DataFrame(knn_imputer.fit_transform(tcga_proteomic_data), columns=tcga_proteomic_data.columns, index =tcga_proteomic_data.index)

tcga_proteomic_data["bcr_patient_barcode"] = tcga_proteomic_t.index.str.split('-', 1).str[1]
tcga_proteomic_data.head()


9171
Duplicate columns found: ['GSN', 'HK1', 'INSR', 'ITGA6', 'TPM1', 'TPM1', 'TPM1', 'HSD17B4', 'PLEC', 'PLEC', 'PLEC', 'PLEC', 'PLEC', 'PLEC', 'SPP1', 'C4BPB', 'P4HA1', 'PLCB4', 'PRKCSH', 'ATP2B4', 'NDUFV3', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'AFAP1L2', 'ATE1', 'ARL6IP4', 'FCGR2B', 'PDE4DIP', 'HNRNPD', 'FBLN2', 'DNM2', 'DNM2', 'DNM2', 'SDCBP', 'C18orf25', 'ENAH', 'MXRA7', 'MTX3', datet

/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['datetime', 'float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['datetime', 'float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['datetime', 'float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got fe

hgnc_symbol,A2M,ACADM,ACADS,ACADVL,ACAT1,PSEN1,ADA,ADSL,AGA,AGT,...,MT-ND2,MT-CO1,MT-CO2,MT-ATP8,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB,bcr_patient_barcode
PNNL-TCGA-09-1664,0.329787,-0.089153,0.170035,0.231383,0.320671,-1.783330,-1.787210,-1.039627,-0.794098,0.625866,...,-0.066533,0.402725,2.093747,-0.170579,0.036133,0.473987,-0.143183,0.204756,0.855386,TCGA-09-1664
PNNL-TCGA-13-1484,-0.629509,0.944590,-0.127556,-0.597598,0.531937,-1.009674,2.102763,0.901624,1.605626,-1.028414,...,-1.358740,-0.702376,0.350870,1.703491,0.870127,0.683079,-0.248484,3.860032,0.028845,TCGA-13-1484
PNNL-TCGA-13-1488,1.207150,0.267773,0.979108,0.403672,0.733048,-2.131350,0.963103,0.713942,-1.502288,0.326415,...,-0.460440,0.269568,0.809037,-0.093694,0.326904,0.658271,0.807531,1.496455,0.428676,TCGA-13-1488
PNNL-TCGA-13-1489,-0.239638,0.675980,0.222113,2.491108,2.108964,0.262226,-0.067715,1.296598,-0.731602,0.122652,...,-0.400554,-0.759737,-1.458776,1.465628,-0.283940,-0.344100,-1.046849,-1.924024,-0.539971,TCGA-13-1489
PNNL-TCGA-13-1494,0.925407,-0.628408,-0.432586,-0.779874,-2.944690,-0.018193,-0.469402,0.431018,0.980886,-0.101015,...,0.415860,0.119793,1.135971,-0.360389,1.483225,0.212162,1.868202,1.029291,-0.026735,TCGA-13-1494


### Load Phospo-proteomic_data

In [11]:
tcga_phospoproteomic = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_Zhang_TCGA_CPTAC_OV/mmc3-2.xlsx",sheet_name="FullOvarianPhosphoproteomeData")
tcga_phospoproteomic.head()



# # transopose and organise 
# tcga_phospoproteomic_t = tcga_phospoproteomic.T
# tcga_phospoproteomic_t.columns = tcga_phospoproteomic_t.iloc[1]
# tcga_phospoproteomic_t = tcga_phospoproteomic_t.iloc[2:]
# # impute 
# # tcga_phospoproteomic_t.index
# knn_imputer = KNNImputer(n_neighbors=5)

# scaler = StandardScaler()
# # Fit and transform the DataFrame
# tcga_phospoproteomic_data = pd.DataFrame(scaler.fit_transform(tcga_phospoproteomic_t), columns=tcga_phospoproteomic_t.columns, index=tcga_phospoproteomic_t.index)

# tcga_phospoproteomic_data = pd.DataFrame(knn_imputer.fit_transform(tcga_phospoproteomic_data), columns=tcga_phospoproteomic_data.columns, index=tcga_phospoproteomic_data.index)

# tcga_phospoproteomic_data["bcr_patient_barcode"] = tcga_phospoproteomic_t.index.str.split('-', 1).str[1]
# tcga_phospoproteomic_data.head()


,Gene,RefSeq,Peptide,Site,AScores,TCGA-13-1484,TCGA-13-1489,TCGA-13-1494,TCGA-13-1499,TCGA-13-2071,...,TCGA-36-2543,TCGA-42-2588,TCGA-42-2590,TCGA-61-1741,TCGA-61-1907,TCGA-61-1919,TCGA-61-1995,TCGA-61-2008,TCGA-61-2095,TCGA-61-2612
0,PSEN1,NP_000012,R.AAVQELSSS*ILAGEDPEER.G,PSEN1-S367s,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PSEN1,NP_000012,R.DS*HLGPHRSTPESR.A,PSEN1-S346s,8.7156,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PSEN1,NP_000012,R.S*LGHPEPLSNGR.P,PSEN1-S43s,90.83,NaN,NaN,NaN,NaN,-0.3370,...,0.3160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PSEN1,NP_000012,R.S*LGHPEPLSNGRPQGNSR.Q,PSEN1-S43s,66.346,-0.411,-1.5,-1.29,-3.58,-0.0734,...,-0.0168,0.988,-0.620,-2.19,0.602,NaN,0.729,-0.0541,NaN,-0.00655
4,PSEN1,NP_000012,R.SLGHPEPLS*NGRPQGNSR.Q,PSEN1-S51s,31.301,NaN,-1.6,NaN,NaN,0.6450,...,-0.1690,NaN,-0.223,-1.68,0.638,-0.0529,NaN,NaN,-1.96,NaN


In [12]:
tcga_phospoproteomic.head()
len(tcga_phospoproteomic) # not in same format as train set... --> maybe best not to use at moment.

42825

In [13]:
merged = pd.merge(tcga_proteomic_data,df_ov_clinical, on='bcr_patient_barcode')
merged.head()

,A2M,ACADM,ACADS,ACADVL,ACAT1,PSEN1,ADA,ADSL,AGA,AGT,...,vital_status,daystodeath_or_LFU,Prog_or_Recur,daystotumorprog-recur-LFU,PlatinumStatus,PlatinumFreeInterval (mos),ethnicity,race,year_of_diagnosis,label
0,-0.629509,0.944590,-0.127556,-0.597598,0.531937,-1.009674,2.102763,0.901624,1.605626,-1.028414,...,DECEASED,3728,Yes,496,Sensitive,10.1,NOT HISPANIC OR LATINO,WHITE,2000,1
1,1.207150,0.267773,0.979108,0.403672,0.733048,-2.131350,0.963103,0.713942,-1.502288,0.326415,...,DECEASED,2154,Yes,351,Sensitive,6.0,NOT HISPANIC OR LATINO,WHITE,2002,1
2,0.282348,0.253173,1.115470,1.145510,0.537811,0.422215,1.826634,0.845054,1.449441,-0.665003,...,DECEASED,2154,Yes,351,Sensitive,6.0,NOT HISPANIC OR LATINO,WHITE,2002,1
3,-0.239638,0.675980,0.222113,2.491108,2.108964,0.262226,-0.067715,1.296598,-0.731602,0.122652,...,DECEASED,2553,Yes,806,Sensitive,17.5,NOT HISPANIC OR LATINO,WHITE,2002,1
4,-0.431909,-0.192069,0.382654,2.501345,1.642925,0.431229,-0.416080,0.906681,-0.368395,-0.461536,...,DECEASED,2553,Yes,806,Sensitive,17.5,NOT HISPANIC OR LATINO,WHITE,2002,1


#### setup HGSOC proteomics data and train classifier with overlapping prots. Then test of tcga data... with label
### first prep dataframe with label slides and signatures.


### Set up dummy learning task to see if there is some signal or corrolation?....

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score


train_df, test_df  = train_test_split(merged, test_size=0.2, random_state=42)

X_train = train_df[intersecting_list]
X_test =  test_df[intersecting_list]
y_train = train_df['label']
y_test = test_df['label']

# Initialize and train the Random Forest classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
rf_pred = rf_clf.predict(X_test)
rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the Random Forest model.
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest ROC AUC: {rf_roc_auc}")

Random Forest Accuracy: 0.7586206896551724
Random Forest ROC AUC: 0.6458333333333333


In [15]:
print(intersecting_list)

['BIRC2', 'LDHA', 'RELA', 'ARNT', 'IDI1', 'TALDO1', 'COX6C', 'CDKN1B', 'KEAP1', 'ACADSB', 'CAMK2D', 'ACOT7', 'HADH', 'TGM2', 'BCL2L1', 'MYD88', 'HACL1', 'AIFM1', 'NBN', 'GLUD1', 'GPX1', 'TFRC', 'NFKB2', 'COX7C', 'GOLPH3L', 'EGFR', 'IDH1', 'COX6A1', 'RAB25', 'RPS6KB2', 'TOP3B', 'NFKB1', 'CASP7', 'IL4I1', 'NCAPH2', 'LGALS1', 'VCAM1', 'CA2', 'CPOX', 'PECR', 'PTGES2', 'ABCB8', 'FDX1', 'XIAP', 'YWHAG', 'LTA4H', 'GPX4', 'RHOA', 'ATM', 'STAT3', 'ATP6V1D', 'COX7A2', 'CHUK', 'PDK1', 'TPMT', 'MTDH', 'RIPK1', 'S100A10', 'NDUFB3', 'PLIN2']


### Merge the wsi slide paths for testing. 

In [16]:
folder_path = '/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides'

# List all filenames in the folder
filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Extract TCGA identifiers from filenames and create a new DataFrame
# Assuming the TCGA ID is always at the start of the filename up to the third '-'
file_df = pd.DataFrame({'Filename': filenames})
file_df['bcr_patient_barcode'] = file_df['Filename'].apply(lambda x: '-'.join(x.split('-')[:3]))

file_df['bcr_patient_barcode']

proteomics_and_wsi = merged.merge(file_df, on='bcr_patient_barcode', how='left')

In [17]:
proteomics_and_wsi["Filename"] = proteomics_and_wsi["Filename"].str[:-4]

In [18]:
len(proteomics_and_wsi)

# now change column namues to be appropriate 

proteomics_and_wsi = proteomics_and_wsi.rename(columns={"bcr_patient_barcode":"case_id","Filename":"slide_id"})




duplicate_columns = check_duplicate_columns(proteomics_and_wsi)
if len(duplicate_columns) > 0:
    print("Duplicate columns found:", duplicate_columns.tolist())
else:
    print("No duplicate columns found.")

# remove dups then count. 


Duplicate columns found: [nan, nan, 'DNM2', 'DNM2', 'SDCBP', 'C4BPB', 'TPM1', 'TPM1', 'TPM1', 'TMPO', 'SORBS1', nan, 'SPP1', nan, nan, nan, nan, 'HNRNPC', nan, nan, 'INSR', 'ITGA6', 'OCIAD1', nan, nan, nan, 'MYO1C', 'CTNND1', 'IMMT', 'HNRNPR', nan, nan, datetime.datetime(2016, 9, 9, 0, 0), 'DBNL', 'GSN', nan, nan, 'SMARCA4', 'PCBP2', 'PCBP2', 'ACTN1', 'EEF1D', 'RUFY3', 'TCOF1', 'POSTN', 'LRRFIP1', 'P4HA1', 'PPHLN1', 'SORBS2', 'SORBS2', nan, 'MTHFSD', 'ARMC10', nan, 'STAU2', 'LDHA', 'AP1B1', 'PALLD', 'KNG1', 'MTX3', 'PGAM5', nan, 'EIF4G2', 'SYK', nan, 'BCAT1', 'ZNF185', 'NPNT', 'PARD3', 'VDAC2', 'PACSIN2', 'FMR1', 'CAST', nan, 'FCGR2B', 'LIMS1', 'RASAL1', 'SPTAN1', nan, 'HSD17B4', nan, 'HMGN3', 'CUX1', 'CAPZB', 'CALU', 'EPB41L2', 'FLNA', nan, nan, 'ATP2B4', datetime.datetime(2016, 9, 7, 0, 0), 'DUT', 'FBLN2', 'HLA-A', 'ITGA7', 'ITIH4', 'LYN', 'MAP2', 'MAP4', nan, 'MYH11', 'PALM', 'PKM', 'PRKCSH', 'PSAP', 'PVRL2', 'RPN2', 'SET', 'SPTAN1', 'TIAL1', 'TMPO', 'TPM4', 'CNBP', 'PPFIBP1', 'KCNA

In [19]:
# proteomics_and_wsi.to_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/TCGA_platinum_reposnce_test_set.csv",index=None)
proteomics_and_wsi.head()

,A2M,ACADM,ACADS,ACADVL,ACAT1,PSEN1,ADA,ADSL,AGA,AGT,...,daystodeath_or_LFU,Prog_or_Recur,daystotumorprog-recur-LFU,PlatinumStatus,PlatinumFreeInterval (mos),ethnicity,race,year_of_diagnosis,label,slide_id
0,-0.629509,0.944590,-0.127556,-0.597598,0.531937,-1.009674,2.102763,0.901624,1.605626,-1.028414,...,3728,Yes,496,Sensitive,10.1,NOT HISPANIC OR LATINO,WHITE,2000,1,TCGA-13-1484-01A-01-TS1.c6df9072-78a4-40f9-a08...
1,-0.629509,0.944590,-0.127556,-0.597598,0.531937,-1.009674,2.102763,0.901624,1.605626,-1.028414,...,3728,Yes,496,Sensitive,10.1,NOT HISPANIC OR LATINO,WHITE,2000,1,TCGA-13-1484-01A-01-BS1.dfebf9da-d2d8-42cd-a94...
2,1.207150,0.267773,0.979108,0.403672,0.733048,-2.131350,0.963103,0.713942,-1.502288,0.326415,...,2154,Yes,351,Sensitive,6.0,NOT HISPANIC OR LATINO,WHITE,2002,1,TCGA-13-1488-01A-01-TS1.3972462a-d30c-473e-a62...
3,1.207150,0.267773,0.979108,0.403672,0.733048,-2.131350,0.963103,0.713942,-1.502288,0.326415,...,2154,Yes,351,Sensitive,6.0,NOT HISPANIC OR LATINO,WHITE,2002,1,TCGA-13-1488-01A-01-BS1.783d5d20-6ac1-4777-9bf...
4,0.282348,0.253173,1.115470,1.145510,0.537811,0.422215,1.826634,0.845054,1.449441,-0.665003,...,2154,Yes,351,Sensitive,6.0,NOT HISPANIC OR LATINO,WHITE,2002,1,TCGA-13-1488-01A-01-TS1.3972462a-d30c-473e-a62...
